In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/ML - pump it/feature_train_full.csv', index_col='id')
train_label = pd.read_csv('/content/drive/MyDrive/ML - pump it/train_label.csv', index_col='id')
test_val = pd.read_csv('/content/drive/MyDrive/ML - pump it/feature_eval_full.csv', index_col='id')

In [ ]:
train_label2 = pd.read_csv('/content/drive/MyDrive/ML - pump it/train_label.csv', index_col='id')

In [ ]:
train_label2.value_counts()

status_group           
functional                 32259
non functional             22824
functional needs repair     4317
dtype: int64

In [ ]:
train_label.head()

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional


In [ ]:
train = MultiColumnLabelEncoder(columns = ['funder','installer','basin','public_meeting', 'permit','extraction_type_class','management_group','payment','water_quality','quantity','source','waterpoint_type' ]).fit_transform(train)
test_val = MultiColumnLabelEncoder(columns = ['funder','installer','basin','public_meeting', 'permit','extraction_type_class','management_group','payment','water_quality','quantity','source','waterpoint_type' ]).fit_transform(test_val)
train_label = MultiColumnLabelEncoder(columns = ['status_group']).fit_transform(train_label)

In [ ]:
train_label.value_counts()

status_group
0               32259
2               22824
1                4317
dtype: int64

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
data_dmatrix = xgb.DMatrix(data=train,label=train_label)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, train_label, test_size=0.2, random_state=123)

In [ ]:
params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

In [ ]:
cv_results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,0.973781,0.002828,0.973860,0.006032
1,0.950248,0.006247,0.950547,0.006394
2,0.928647,0.008499,0.929163,0.008583
3,0.908104,0.005554,0.908704,0.005776
4,0.893777,0.008991,0.894645,0.003498


In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(max_depth=13,n_estimators=200,learning_rate=0.075)
xgb_model.fit(X_train,y_train)
predictions = xgb_model.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
preds = pd.DataFrame(predictions)

In [ ]:
y_test

,status_group
id,
2369,0
43734,2
17667,1
56384,2
45781,0
...,...
69099,0
150,0
68205,1


In [ ]:
preds.index = y_test.index

In [ ]:
preds

,0
id,
2369,0
43734,2
17667,0
56384,2
45781,0
...,...
69099,0
150,0
68205,2


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, preds)

0.8024410774410774

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(max_depth=13,n_estimators=200,learning_rate=0.075)
xgb_model.fit(train,train_label)
predictions = xgb_model.predict(test_val)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
preds = pd.DataFrame(predictions)

In [ ]:
preds.index = test_val.index

In [ ]:
preds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14850 entries, 50785 to 68707
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   status_group  14850 non-null  int64
dtypes: int64(1)
memory usage: 232.0 KB


In [ ]:
preds = preds.rename(columns={0: 'status_group'})

In [ ]:
preds['status_group'] = preds['status_group'].replace({1:'functional needs repair', 0:'functional',2:'non functional'})

In [ ]:
preds.head()

,status_group
id,
50785,non functional
51630,functional
17168,functional
45559,non functional
49871,functional


In [ ]:
test_val.head()

,amount_tsh,funder,gps_height,installer,basin,region_code,district_code,population,public_meeting,permit,extraction_type_class,management_group,payment,water_quality,quantity,source,waterpoint_type,year
id,,,,,,,,,,,,,,,,,,
50785,0.0,175,1996,214,0,21,3,321,1,1,3,2,0,6,3,5,6,6
51630,0.0,250,1569,221,5,2,2,300,1,1,0,4,0,6,2,8,1,4
17168,0.0,250,1567,221,0,13,2,500,1,1,3,4,0,6,2,5,6,5
45559,0.0,222,267,262,7,80,43,250,1,1,3,4,6,6,0,7,6,3
49871,500.0,73,1260,75,7,10,3,60,1,1,0,4,3,6,1,8,1,4


In [ ]:
preds.to_csv('/content/drive/MyDrive/ML - pump it/SubmissionFormat.csv')